# BIZ 프로젝트 : 부실기업 예측

## Step2 : 데이터 전처리

부실기업은 과거 3년간 연속해 이자보상 배율이 1.0미만인 기업을 의미하며  
이를 이용하여 타겟변수인 부실기업여부 변수를 생성

### 필요라이브러리 불러오기

In [29]:
import pandas as pd
import numpy as np

### 데이터 불러오기

In [30]:
RANDOM_STATE = 110

data = pd.read_csv("../../project/data/clean_financial_data_eda.csv", encoding='utf-8-sig', low_memory=False)

In [31]:
data.columns

Index(['업체코드', '종목명', '2020/총자산증가율', '2021/총자산증가율', '2022/총자산증가율',
       '2023/총자산증가율', '2020/유형자산증가율', '2021/유형자산증가율', '2022/유형자산증가율',
       '2023/유형자산증가율',
       ...
       '2021/조세공과(구성비)', '2022/조세공과(구성비)', '2023/조세공과(구성비)', '2020/감가상각비(구성비)',
       '2021/감가상각비(구성비)', '2022/감가상각비(구성비)', '2023/감가상각비(구성비)', '2022/부실기업',
       '2023/부실기업', '종목코드_유무'],
      dtype='object', length=437)

In [32]:
data.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38639 entries, 0 to 38638
Data columns (total 437 columns):
 #    Column                     Non-Null Count  Dtype  
---   ------                     --------------  -----  
 0    업체코드                       38639 non-null  object 
 1    종목명                        38639 non-null  object 
 2    2020/총자산증가율                38639 non-null  float64
 3    2021/총자산증가율                38639 non-null  float64
 4    2022/총자산증가율                38639 non-null  float64
 5    2023/총자산증가율                38639 non-null  float64
 6    2020/유형자산증가율               38639 non-null  float64
 7    2021/유형자산증가율               38639 non-null  float64
 8    2022/유형자산증가율               38639 non-null  float64
 9    2023/유형자산증가율               38639 non-null  float64
 10   2020/유동자산증가율               38639 non-null  float64
 11   2021/유동자산증가율               38639 non-null  float64
 12   2022/유동자산증가율               38639 non-null  float64
 13   2023/유동자산증가율               38

C:\Users\juneh\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: FutureWarning: null_counts is deprecated. Use show_counts instead
  """Entry point for launching an IPython kernel.


### 이후 할것 스케치

전처리(변수선택)
- 후진제거법
- t-검정

모델링
- 20,21년도 변수 -> 22년 부실기업 여부 예측
- 20,21,22년도 변수 -> 23년도 부실기업 여부 예측

In [35]:
import pandas as pd
import statsmodels.api as sm
from scipy.stats import ttest_ind
from statsmodels.stats.outliers_influence import variance_inflation_factor

# 종속 변수는 '2023/부실기업'으로 가정
y = data['2023/부실기업']
X = data.drop(columns=['2022/부실기업', '2023/부실기업', '업체코드', '종목명'])

# 상수항 추가
X = sm.add_constant(X)

# t-검정을 통해 변수 선택
def t_test_selection(X, y, significance_level=0.05):
    selected_vars = []
    for column in X.columns:
        if column == 'const':
            continue
        group1 = X[y == 0][column]
        group2 = X[y == 1][column]
        t_stat, p_value = ttest_ind(group1, group2)
        if p_value < significance_level:
            selected_vars.append(column)
    return selected_vars

# t-검정을 통해 선택된 변수들
selected_vars_ttest = t_test_selection(X, y)

# 선택된 변수들로 새로운 데이터프레임 생성
X_selected_ttest = X[selected_vars_ttest]

# 결과 확인
print("\n선택된 변수들로 구성된 데이터프레임:")
print(X_selected_ttest.columns)


선택된 변수들로 구성된 데이터프레임:
Index(['2020/유형자산증가율', '2021/유동자산증가율', '2022/자기자본증가율', '2023/자기자본증가율',
       '2020/영업이익증가율', '2022/영업이익증가율', '2021/법인세비용차감전순이익증가율', '2021/순이익증가율',
       '2021/종업원수증가율', '2020/총자본영업이익율',
       ...
       '2023/기계투자효율', '2021/부가가치율', '2023/부가가치율', '2022/노동소득분배율',
       '2022/인건비(구성비)', '2021/임차료(구성비)', '2022/임차료(구성비)', '2021/조세공과(구성비)',
       '2022/감가상각비(구성비)', '종목코드_유무'],
      dtype='object', length=203)


In [36]:
# 다중공선성 제거를 위한 VIF 계산
def calculate_vif(X):
    vif = pd.DataFrame()
    vif["Variable"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return vif

# VIF가 높은 변수 제거
def remove_high_vif(X, threshold=10):
    while True:
        vif = calculate_vif(X)
        max_vif = vif['VIF'].max()
        if max_vif > threshold:
            excluded_var = vif.loc[vif['VIF'] == max_vif, 'Variable'].values[0]
            X = X.drop(columns=[excluded_var])
        else:
            break
    return X

# VIF가 높은 변수 제거
X_selected_ttest_vif = remove_high_vif(X_selected_ttest)

c:\Users\juneh\AppData\Local\Programs\Python\Python37\lib\site-packages\statsmodels\stats\outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


KeyboardInterrupt: 

In [ ]:
# 로지스틱 회귀분석 및 후진선택법 적용
def backward_elimination_logit(X, y, significance_level=0.05):
    X = sm.add_constant(X)
    num_vars = len(X.columns)
    for i in range(num_vars):
        logit_model = sm.Logit(y, X).fit(disp=0)
        max_p_value = max(logit_model.pvalues)
        if max_p_value > significance_level:
            excluded_var = logit_model.pvalues.idxmax()
            X = X.drop(columns=[excluded_var])
        else:
            break
    return X

# 후진선택법을 통해 선택된 변수들
X_selected_logit = backward_elimination_logit(X_selected_ttest_vif, y)

# 선택된 변수들 출력
print("\n후진선택법을 통해 선택된 변수들:")
print(X_selected_logit.columns)

---

### step1. 후진제거법

In [12]:
import pandas as pd
import statsmodels.api as sm

# 종속 변수는 '2023/부실기업'으로 가정
y = data['2023/부실기업']
X = data.drop(columns=['2022/부실기업', '2023/부실기업', '업체코드', '종목명'])

# 상수항 추가
X = sm.add_constant(X)

# 후진 제거법 적용
def backward_elimination(X, y, significance_level=0.05):
    num_vars = len(X.columns)
    for i in range(num_vars):
        regressor_OLS = sm.OLS(y, X).fit()
        max_p_value = max(regressor_OLS.pvalues)
        if max_p_value > significance_level:
            excluded_var = regressor_OLS.pvalues.idxmax()
            X = X.drop(columns=[excluded_var])
        else:
            break
    return X

# 후진 제거법을 통해 선택된 변수들
X_selected = backward_elimination(X, y)

# 선택된 변수들 출력
print("선택된 변수들:")
print(X_selected.columns)

선택된 변수들:
Index(['const', '2021/총자산증가율', '2022/총자산증가율', '2023/총자산증가율', '2023/유동자산증가율',
       '2021/자기자본증가율', '2023/매출액증가율', '2023/순이익증가율', '2021/종업원수증가율',
       '2020/총자본영업이익율',
       ...
       '2023/임차료(구성비)', '2020/조세공과(구성비)', '2021/조세공과(구성비)', '2022/조세공과(구성비)',
       '2023/조세공과(구성비)', '2020/감가상각비(구성비)', '2021/감가상각비(구성비)',
       '2022/감가상각비(구성비)', '2023/감가상각비(구성비)', '종목코드_유무'],
      dtype='object', length=155)


### step2. t-검정

In [ ]:
import pandas as pd
import statsmodels.api as sm
from scipy.stats import ttest_ind

# t-검정을 통해 변수 선택
def t_test_selection(X, y, significance_level=0.05):
    selected_vars = []
    for column in X.columns:
        if column == 'const':
            continue
        group1 = X[y == 0][column]
        group2 = X[y == 1][column]
        t_stat, p_value = ttest_ind(group1, group2)
        if p_value < significance_level:
            selected_vars.append(column)
    return selected_vars

# t-검정을 통해 선택된 변수들
selected_vars_ttest = t_test_selection(X_selected, y)

# 선택된 변수들 출력
print("t-검정을 통해 선택된 변수들:")
print(selected_vars_ttest)

# 선택된 변수들로 새로운 데이터프레임 생성
X_selected_ttest = X_selected[selected_vars_ttest]

# 결과 확인
print("\n선택된 변수들로 구성된 데이터프레임:")
print(X_selected_ttest.columns)

t-검정을 통해 선택된 변수들:
['2021/종업원수증가율', '2020/총자본영업이익율', '2021/총자본영업이익율', '2022/총자본영업이익율', '2023/총자본영업이익율', '2022/총자본법인세비용차감전순이익율', '2023/총자본법인세비용차감전순이익율', '2023/총자본순이익율', '2021/기업법인세비용차감전순이익율', '2022/기업법인세비용차감전순이익율', '2023/기업법인세비용차감전순이익율', '2021/기업순이익율', '2022/기업순이익율', '2021/자기자본법인세비용차감전순이익율', '2022/자기자본법인세비용차감전순이익율', '2023/자기자본법인세비용차감전순이익율', '2021/자기자본순이익율', '2022/자기자본순이익율', '2023/자기자본순이익율', '2020/매출액법인세비용차감전순이익율', '2021/매출액순이익율', '2020/매출액총이익율', '2021/매출액총이익율', '2022/매출액총이익율', '2023/매출액총이익율', '2020/매출액영업이익율', '2020/수지비율', '2021/매출원가율', '2021/감가상각율', '2021/감가상각비/총비용비율', '2021/조세/조세차감전순이익비율', '2022/금융비용/총부채', '2021/금융비용/총비용비율', '2022/금융비용/총비용비율', '2023/금융비용/총비용비율', '2020/금융비용/매출액비율', '2021/영업이익이자보상비율', '2021/대출효율성계수(법인세비용차감전순이익)', '2023/대출효율성계수(법인세비용차감전순이익)', '2023/EBITDA대매출액', '2023/EBITDA대금융비용', '2020/자기자본비율', '2022/자기자본비율', '2021/유동비율', '2021/당좌비율', '2021/현금비율', '2021/비유동자산장기적합율', '2023/부채비율', '2021/유동부채비율', '2023/비유동부채비율', '2022/차입금의존도', '2023/차입금의존도', '2021/차입금/자기자본', '2022/차입금/매출액비율'

In [ ]:
# 결과 확인
print("\n선택된 변수들로 구성된 데이터프레임:")
print(X_selected_ttest.columns)


선택된 변수들로 구성된 데이터프레임:
Index(['2021/종업원수증가율', '2020/총자본영업이익율', '2021/총자본영업이익율', '2022/총자본영업이익율',
       '2023/총자본영업이익율', '2022/총자본법인세비용차감전순이익율', '2023/총자본법인세비용차감전순이익율',
       '2023/총자본순이익율', '2021/기업법인세비용차감전순이익율', '2022/기업법인세비용차감전순이익율',
       '2023/기업법인세비용차감전순이익율', '2021/기업순이익율', '2022/기업순이익율',
       '2021/자기자본법인세비용차감전순이익율', '2022/자기자본법인세비용차감전순이익율',
       '2023/자기자본법인세비용차감전순이익율', '2021/자기자본순이익율', '2022/자기자본순이익율',
       '2023/자기자본순이익율', '2020/매출액법인세비용차감전순이익율', '2021/매출액순이익율', '2020/매출액총이익율',
       '2021/매출액총이익율', '2022/매출액총이익율', '2023/매출액총이익율', '2020/매출액영업이익율',
       '2020/수지비율', '2021/매출원가율', '2021/감가상각율', '2021/감가상각비/총비용비율',
       '2021/조세/조세차감전순이익비율', '2022/금융비용/총부채', '2021/금융비용/총비용비율',
       '2022/금융비용/총비용비율', '2023/금융비용/총비용비율', '2020/금융비용/매출액비율',
       '2021/영업이익이자보상비율', '2021/대출효율성계수(법인세비용차감전순이익)',
       '2023/대출효율성계수(법인세비용차감전순이익)', '2023/EBITDA대매출액', '2023/EBITDA대금융비용',
       '2020/자기자본비율', '2022/자기자본비율', '2021/유동비율', '2021/당좌비율', '2021/현금비율',
       '2021/비유동자산장기적합율',

In [23]:
# 2020 들어간 변수 필터링
cols_2020 = X_selected_ttest.filter(like='2020').columns
print("\n2020이 들어간 변수들:")
for col in cols_2020:
    print(col)


2020이 들어간 변수들:
2020/총자본영업이익율
2020/매출액법인세비용차감전순이익율
2020/매출액총이익율
2020/매출액영업이익율
2020/수지비율
2020/금융비용/매출액비율
2020/자기자본비율
2020/총자본회전율
2020/노동장비율(종업원1인당)
2020/자본집약도(종업원1인당)


In [27]:
# 2021 들어간 변수 필터링
cols_2021 = X_selected_ttest.filter(like='2021').columns
print("\n2021이 들어간 변수들:")
for col in cols_2021:
    print(col)


2021이 들어간 변수들:
2021/종업원수증가율
2021/총자본영업이익율
2021/기업법인세비용차감전순이익율
2021/기업순이익율
2021/자기자본법인세비용차감전순이익율
2021/자기자본순이익율
2021/매출액순이익율
2021/매출액총이익율
2021/매출원가율
2021/감가상각율
2021/감가상각비/총비용비율
2021/조세/조세차감전순이익비율
2021/금융비용/총비용비율
2021/영업이익이자보상비율
2021/대출효율성계수(법인세비용차감전순이익)
2021/유동비율
2021/당좌비율
2021/현금비율
2021/비유동자산장기적합율
2021/유동부채비율
2021/차입금/자기자본
2021/비유동부채/순운전자본비율
2021/사내유보율
2021/유보액/총자산비율
2021/총C/F대총자본비율
2021/총자본회전율
2021/매출액(종업원1인당)
2021/부가가치율
2021/임차료(구성비)
2021/조세공과(구성비)


In [25]:
# 2022 들어간 변수 필터링
cols_2022 = X_selected_ttest.filter(like='2022').columns
print("\n2022가 들어간 변수들:")
for col in cols_2022:
    print(col)


2022가 들어간 변수들:
2022/총자본영업이익율
2022/총자본법인세비용차감전순이익율
2022/기업법인세비용차감전순이익율
2022/기업순이익율
2022/자기자본법인세비용차감전순이익율
2022/자기자본순이익율
2022/매출액총이익율
2022/금융비용/총부채
2022/금융비용/총비용비율
2022/자기자본비율
2022/차입금의존도
2022/차입금/매출액비율
2022/비유동부채/순운전자본비율
2022/적립금비율
2022/사내유보율
2022/유보액/총자산비율
2022/총C/F대총자본비율
2022/노동장비율(종업원1인당)
2022/자본집약도(종업원1인당)
2022/인건비(구성비)
2022/임차료(구성비)
2022/감가상각비(구성비)


In [28]:
# 2023 들어간 변수 필터링
cols_2023 = X_selected_ttest.filter(like='2023').columns
print("\n2023가 들어간 변수들:")
for col in cols_2023:
    print(col)


2023가 들어간 변수들:
2023/총자본영업이익율
2023/총자본법인세비용차감전순이익율
2023/총자본순이익율
2023/기업법인세비용차감전순이익율
2023/자기자본법인세비용차감전순이익율
2023/자기자본순이익율
2023/매출액총이익율
2023/금융비용/총비용비율
2023/대출효율성계수(법인세비용차감전순이익)
2023/EBITDA대매출액
2023/EBITDA대금융비용
2023/부채비율
2023/비유동부채비율
2023/차입금의존도
2023/비유동부채/순운전자본비율
2023/순운전자본/총자본비율
2023/사내유보율
2023/유보액/총자산비율
2023/총C/F대총자본비율
2023/총자본회전율
2023/매출액(종업원1인당)
2023/순이익(종업원1인당)
2023/자본집약도(종업원1인당)


.